In [2]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 47.0 MB/s eta 0:00:00


In [20]:
import fitz  # PyMuPDF

file_path = 'canto1_ch1.pdf'

pdf_document = fitz.open(file_path)

all_data = []

for page_num in range(len(pdf_document)):
  page_data = []
  page = pdf_document.load_page(page_num)
  text_instances = page.get_text("dict")  # Get text as a list of dictionaries

  for block in text_instances["blocks"]:
    for line in block["lines"]:
      for span in line["spans"]:
        # Keep only "size", "font", and "text" keys
        span_data = {
            "size": int(span["size"]),  # Convert "size" to integer
            "font": span["font"],
            "color": span["color"] > 0,
            "text": span["text"],
        }
        page_data.append(span_data)

  page_data = page_data[:-3]
  all_data.extend(page_data)

print(len(all_data))
pdf_document.close()


2069


In [ ]:
unique_fonts = set()

# Extract unique font names from dictionaries in all_data
for entry in all_data:
  font_name = entry["font"]
  unique_fonts.add(font_name)

print(unique_fonts)

{'ChanakyaItalic', 'ChanakyaBold', 'TimesNewRomanPS-BoldMT', 'Chanakya', 'Arial-BoldItalicMT'}


In [44]:
# Create HTML content
html_content = "<!DOCTYPE html>\n<html>\n<head>\n"
html_content += '<link rel="stylesheet" type="text/css" href="styles.css">\n'
html_content += '<style>@font-face {font-family: "Chanakya";src: url("chanakya.ttf");}</style>'
html_content += "</head>\n<body>\n<p style=\"font-family: Chanakya;\">"

# Add text with custom font to HTML content
prev_font_size = None
for i, entry in enumerate(all_data):
    font_size = entry["size"]
    text = entry["text"]
    is_colored = entry.get("color", 0)  # Default to 0 if "color" key doesn't exist
    color = f'rgb(0, 0, {255 if is_colored else 0})'
    font_name = entry["font"]
    is_bold = "bold" in font_name.lower()
    is_italic = "italic" in font_name.lower()

    if (i == len(all_data) - 1) or (i == 0):
        html_content += "<br>"

    # Add <br> tag before each span when the font size is anything other than 11 and 15
    elif font_size != 11 and font_size != 15:
        html_content += "<br>"

    # Add <br> tag if font size is 11 or 15 and previous and next entries also have the same font size
    elif font_size == 11 and (all_data[i + 1]["size"] != 11 or all_data[i - 1]["size"] != 11):
        html_content += "<br>"
    elif font_size == 15 and (all_data[i + 1]["size"] != 15 or all_data[i - 1]["size"] != 15):
        html_content += "<br>"

    # Add text with appropriate styling
    if is_bold:
        html_content += f'<span style="font-weight: bold; font-size: {font_size}px; color: {color};">{text}</span>'
    elif is_italic:
        html_content += f'<span style="font-style: italic; font-size: {font_size}px; color: {color};">{text}</span>'
    else:
        html_content += f'<span style="font-size: {font_size}px; color: {color};">{text}</span>'

    prev_font_size = font_size

html_content += "</p>\n</body>\n</html>"

# Write HTML content to file
with open("output.html", "w") as html_file:
    html_file.write(html_content)

print("HTML generated successfully!")


HTML generated successfully!
